You will need cuml for this--you can of course cluster embeddings with `sklearn` or other libraries, but for this project we used cuml because of its relatively fast speed.

See: [https://docs.rapids.ai/install#selector](https://docs.rapids.ai/install#selector) for instructions on how to install cuml.

For the 24.6 version of the NVIDIA RAPIDS suite (which cuml is a component of), here is the install from the selector:

```
pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==24.6.*
```

The code in this notebook assumes you have generated embeddings from a trained model; here we'll use embeddings from the model we trained on the ABC AIBS MERFISH data. To download this data see the script in `scripts/training/download_aibs.sh` or visit the ABC Atlas website [https://portal.brain-map.org/atlases-and-data/bkp/abc-atlas](https://portal.brain-map.org/atlases-and-data/bkp/abc-atlas).

This is minimal code to smooth embeddings generated from a trained model along the spatial cell graph. We basically smooth cells using a weighted average of their neighbors. The weighting function is a simple gaussian kernel with a fixed spatial FWHM/sigma. 

This can be very slow, so in order to improve the speed of the workflow we use cuml's approximate nearest neighbor code to look at the nearest $n$ neighbors and only smooth over those cells. 

In [15]:
import torch
import numpy as np
import cuml
import pandas as pd
import anndata as ad
import tqdm

In [25]:
x = torch.load(
    "/home/ajl/work/d2/code/mousebrain/clustering/embeddings/temp-for-2mmc/2mmc_epoch39_all-cat-sm40_rad17.pth"
)

# x = np.load('embeds.npy')

In [3]:
df = pd.read_csv(
    "/home/ajl/work/d1/abc/metadata/MERFISH-C57BL6J-638850-CCF/20231215/cell_metadata_with_parcellation_annotation.csv"
)
adata = ad.read_h5ad('/home/ajl/work/d1/abc/expression_matrices/MERFISH-C57BL6J-638850/20230630/C57BL6J-638850-log2.h5ad')


In [4]:
df = df[df.cell_label.isin(adata.obs.index)]

In [33]:
df = df.reset_index(drop=True) # we need a contiguous index for next steps

In [19]:
def gaussian(d, sigma):
    return torch.exp(-d**2 / (2 * sigma**2))

In [8]:
df['x_reconstructed_scaled'] = np.round(df['x_reconstructed'] * 100)
df['y_reconstructed_scaled'] = np.round(df['y_reconstructed'] * 100)


In [13]:
p = 2 # at the scale we are using e ach "voxel" is a non-isotropic 10micron 
# "xy" resolution box 
sigma = np.sqrt(2 * p**2)*0.5 / (2*np.sqrt(2*np.log(2)))
print(sigma)

6.005612043932249


In [34]:
n_neighbors = 1000
all_embeds = []
for uniq_section in tqdm.tqdm(df.brain_section_label.unique()):
	smoothed_embeds = []
	section = df[df.brain_section_label==uniq_section]
	positions = section[['x_reconstructed_scaled', 'y_reconstructed_scaled']].values

	nearest = cuml.NearestNeighbors(n_neighbors=n_neighbors)
	nearest.fit(positions) 
	distances, indices = nearest.kneighbors(positions)

	for idx, dist in zip(indices, distances):
		dist = torch.from_numpy(dist)
		weights = gaussian(dist, sigma)
		normalized_wts = weights / weights.sum()

		indices_X = section.index[idx]
		embeds_neighbors = x[indices_X]
		weighted = embeds_neighbors * normalized_wts[:, None]
		embeds_weighted = weighted.sum(axis=0)
		smoothed_embeds.append(embeds_weighted)
		
	smoothed_embeds_cat = torch.stack(smoothed_embeds)
	# highly advised to save this and not keep all of them in memory
	#torch.save(smoothed_embeds_cat, f'{uniq_section}_smoothed.pth')
	all_embeds.append(smoothed_embeds_cat)

100%|██████████| 53/53 [00:41<00:00,  1.27it/s]
